In general, enforcing schema on existing collection would requires more works because it could return invalid documents and we need to figure these set of documents in a way to match the new schema. 

## Case 1: Applying on a new collection

Enforcing schema on a new collection is easy. We just need to setup the rules of inserting documents before we insert any documents. Invalid documents cannot be passed into the collection: 

In [ ]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017')
db = client['mydatabase']

# Define a validation schema for embedded many-to-many relationship
validation_schema = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["name", "related_documents"],
        "properties": {
            "name": {
                "bsonType": "string",
                "description": "The name of the document"
            },
            "related_documents": {
                "bsonType": "array",
                "description": "An array of related documents",
                "items": {
                    "bsonType": "object",
                    "required": ["document_id"],
                    "properties": {
                        "document_id": {
                            "bsonType": "objectId",
                            "description": "The ObjectId of the related document"
                        },
                        "relationship_data": {
                            "bsonType": "object",
                            "description": "Additional relationship data"
                        }
                    }
                }
            }
        }
    }
}

# Create the collection with the validation schema
collection = db.create_collection('mycollection', validator=validation_schema)

# Enable strict validation
collection.options().update({'validationLevel': 'strict', 'validationAction': 'error'})

# Insert a document with embedded related documents
document = {
    "name": "Document 1",
    "related_documents": [
        {
            "document_id": ObjectId("5fe83b3b11a41e4b1f6a1a4d"),
            "relationship_data": {"field1": "value1"}
        },
        {
            "document_id": ObjectId("5fe83b3b11a41e4b1f6a1a4e"),
            "relationship_data": {"field2": "value2"}
        }
    ]
}

# Insert the document
collection.insert_one(document)

# Case 2: Applying on existing collection

## Enfocing "referencing" in some fields in a collection

#### Step 1: Define your relationship model

Determine whether you want to use embedded documents or references for your relationships. Embedded documents store related data within the same document, while referencing involves storing references to related documents.

In [ ]:
db.createCollection("myCollection", {
   validator: {
      $jsonSchema: {
         bsonType: "object",
         required: ["referenceField"],
         properties: {
            referenceField: {
               bsonType: "objectId",
               description: "Reference to another document"
            }
         }
      }
   }
})

#### Step 2: Enable the defined validation schema

We can do this by setting the validator option when creating the collection or by updating an existing collection.

In [ ]:
db.runCommand({
   collMod: "myCollection",
   validator: {
      $jsonSchema: {
         // Define your validation schema here
      }
   },
   validationLevel: "strict",
   validationAction: "error"
})